In [2]:
import os
import cv2
from tqdm import tqdm
import numpy as np
import scipy
import open3d as o3d
import read_raw_file as RRF
import preprocess as PRE
import copy


In [85]:
path = r'D:\StageE23\Data\Ete_2022\Participant06\autocorrection\Prise02'
save_path1 = path + '/intensity_removed_bg/'
save_path2 = path + '/xyz_removed_bg/'
intensity_path = path + '/intensity/'
xyz_path = path + '/xyz/'

os.makedirs(save_path1, exist_ok=True)
os.makedirs(save_path2, exist_ok=True)

for filename in tqdm(os.listdir(intensity_path)):
        index_I = filename.find('_I') + 1
        xyz_file = filename[:index_I] + 'XYZ_' + filename[index_I+2:-4] + '.raw'
        image_path = os.path.join(intensity_path, filename)
        pc_path = os.path.join(xyz_path, xyz_file)
        xyz = PRE.read_single_xyz_raw_file(pc_path)
        image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
        
        removed_bg_points, image_no_bg = PRE.remove_bg(xyz, image)
        
        save_image_path = os.path.join(save_path1, filename)
        cv2.imwrite(save_image_path, image_no_bg)

        pc_removed_bg = o3d.geometry.PointCloud()
        pc_removed_bg.points = o3d.utility.Vector3dVector(removed_bg_points)
        cl, ind = pc_removed_bg.remove_statistical_outlier(nb_neighbors=20,
                                                std_ratio=2.0)
        inlier_removed_bg = pc_removed_bg.select_by_index(ind)

        save_xyz_path = os.path.join(save_path2, xyz_file[:-4] + '.ply')
        o3d.io.write_point_cloud(save_xyz_path, inlier_removed_bg)

100%|██████████| 79/79 [01:29<00:00,  1.13s/it]


In [46]:
pc0 = o3d.io.read_point_cloud(r'D:\StageE23\Data\Ete_2022\Participant06\autocorrection\Prise01\xyz_removed_bg\auto_01_007367_XYZ_0.ply')
pc1 = o3d.io.read_point_cloud(r'D:\StageE23\Data\Ete_2022\Participant06\autocorrection\Prise01\xyz_removed_bg\auto_01_007368_XYZ_1.ply')
pc40 = o3d.io.read_point_cloud(r'D:\StageE23\Data\Ete_2022\Participant06\autocorrection\Prise01\xyz_removed_bg\auto_01_007427_XYZ_40.ply')
pc72 = o3d.io.read_point_cloud(r'D:\StageE23\Data\Ete_2022\Participant06\autocorrection\Prise01\xyz_removed_bg\auto_01_007481_XYZ_72.ply')

o3d.visualization.draw_geometries([pc0])

In [53]:
def crop_pc(pc):
    pc_array = np.asarray(pc.points)

    legs = np.where(pc_array[:,0] < -200)
    head = np.where(pc_array[:,0] > np.max(pc_array[:,0])-280)
    armR = np.where(pc_array[:,1] > np.max(pc_array[:,1])-110)
    armL = np.where(pc_array[:,1] < np.min(pc_array[:,1])+110)

    mask = np.ones((pc_array.shape[0],3), dtype=bool)
    mask[legs] = False
    mask[head] = False
    mask[armR] = False
    mask[armL] = False
    #mask.reshape((500123, 3))

    back = np.array(pc_array[mask], dtype=np.float64)

    pc = np.array(back.reshape((back.shape[0]//3, 3)))

    pc_cropped = o3d.geometry.PointCloud()
    pc_cropped.points = o3d.utility.Vector3dVector(pc)
    
    return pc_cropped

pc_center = crop_pc(pc0)
o3d.visualization.draw_geometries([pc_center])


In [54]:
def draw_registration_result(source, target, transformation):
    source_temp = copy.deepcopy(source)
    target_temp = copy.deepcopy(target)
    source_temp.paint_uniform_color([1, 0.706, 0])
    target_temp.paint_uniform_color([0, 0.651, 0.929])
    source_temp.transform(transformation)
    o3d.visualization.draw_geometries([source_temp, target_temp])

In [66]:
body0 = crop_pc(pc0)
body1 = crop_pc(pc1)
body40 = crop_pc(pc40)
body72 = crop_pc(pc72)

trans_init = np.identity(4)

icp1 = o3d.pipelines.registration.registration_icp(body0, body1, 10, trans_init, o3d.pipelines.registration.TransformationEstimationPointToPoint())
print(icp1.transformation)

icp40 = o3d.pipelines.registration.registration_icp(body0, body40, 10, trans_init, o3d.pipelines.registration.TransformationEstimationPointToPoint())
print(icp40.transformation)

draw_registration_result(body0, body40, icp40.transformation)

[[ 9.99999998e-01  4.90122558e-05  3.66597539e-05 -6.65848782e-02]
 [-4.88960038e-05  9.99994992e-01 -3.16441350e-03  6.01753542e+00]
 [-3.68146654e-05  3.16441170e-03  9.99994993e-01 -4.08602392e-01]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]
[[ 9.99990322e-01  3.26052401e-03  2.95392065e-03 -8.23755694e+00]
 [-2.59254330e-03  9.79148998e-01 -2.03126854e-01  4.00921721e+02]
 [-3.55462844e-03  2.03117230e-01  9.79147974e-01  3.45915971e+01]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]


In [80]:
RST = icp40.transformation
T = RST[0:3, 3]
print(T)
len_T = np.linalg.norm(T)
print(f'Distance de translation : {len_T} mm')
RS = RST[0:3, 0:3]
print(RS)
S = np.zeros((3,3))
S[0,0] = np.linalg.norm(RS[:,0])
S[1,1] = np.linalg.norm(RS[:,1])
S[2,2] = np.linalg.norm(RS[:,2])
print(S)
R = np.matmul(RS, np.linalg.inv(S)) #S est la matric identité, donc R=RS, aucun scaling
print(R)

[ -8.23755694 400.92172093  34.59159705]
Distance de translation : 402.4955431318786 mm
[[ 0.99999032  0.00326052  0.00295392]
 [-0.00259254  0.979149   -0.20312685]
 [-0.00355463  0.20311723  0.97914797]]
[[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]
[[ 0.99999032  0.00326052  0.00295392]
 [-0.00259254  0.979149   -0.20312685]
 [-0.00355463  0.20311723  0.97914797]]


In [84]:
Tr = np.trace(R)
cos_rot = (Tr-1)/2
angle_rot = np.degrees(np.arccos(cos_rot))
print(angle_rot)

axis_rot = np.multiply(1/np.sqrt((3-Tr)*(1+Tr)), np.array([R[2,1]-R[1,2], R[0,2]-R[2,0], R[1,0]-R[0,1]]))
print(axis_rot)

11.722346017916534
[ 0.99976795  0.01601756 -0.01440442]
